In [ ]:
import os
import requests
import pandas as pd
from google.colab import drive
from googleapiclient.discovery import build
from tqdm.notebook import tqdm # Para barras de progreso en Colab

In [ ]:
# Monto Google Drive para guardar los archivos
print("Montando Google Drive...")
drive.mount('/content/drive')

# Defino la ruta donde se guardarán las fuentes y el dataset CSV

# Se puede cambiar esta ruta a tu preferencia
OUTPUT_DIR = '/content/drive/MyDrive/ocr_fonts'
print(f"Las fuentes se guardarán en: {OUTPUT_DIR}")

# Creo el directorio de salida si no existe
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Directorio '{OUTPUT_DIR}' asegurado.")



Montando Google Drive...
Mounted at /content/drive
Las fuentes se guardarán en: /content/drive/MyDrive/ocr_fonts
Directorio '/content/drive/MyDrive/ocr_fonts' asegurado.


In [ ]:

# Se define la ruta donde se guardarán las fuentes y el dataset CSV
OUTPUT_DIR = '/content/drive/MyDrive/ocr_fonts'
print(f"Las fuentes se guardarán en: {OUTPUT_DIR}")

# Creo el directorio de salida si no existe
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Directorio '{OUTPUT_DIR}' asegurado.")

# Configuración de la API de Google Fonts
GOOGLE_FONTS_API_KEY = "Ingresar_Clave_API"

if GOOGLE_FONTS_API_KEY == "TU_CLAVE_API_DE_GOOGLE_FONTS_AQUI":
    raise ValueError("Por favor, reemplaza 'TU_CLAVE_API_DE_GOOGLE_FONTS_AQUI' con tu clave de API de Google Fonts.")

# Construyo el servicio de la API de Google Fonts
service = build('webfonts', 'v1', developerKey=GOOGLE_FONTS_API_KEY)

# Obtengo y Filtro Fuentes de Google Fonts

print("\nObteniendo lista de fuentes de Google Fonts (esto puede tardar unos segundos)...")
try:
    # Obtengo todas las fuentes disponibles
    request = service.webfonts().list()
    response = request.execute()
    all_fonts = response.get('items', [])
    print(f"Total de fuentes disponibles en la API: {len(all_fonts)}")
except Exception as e:
    print(f"Error al conectar con la API de Google Fonts: {e}")
    print("Asegúrate de que tu clave de API sea válida y que tengas conexión a internet.")
    all_fonts = [] # Me aseguro que la lista esté vacía en caso de error

# Filtros para asegurar una buena cobertura y variedad
# Considera las categorías de fuentes
# Se han añadido más categorías para una mayor diversidad.
font_categories = ['sans-serif', 'serif', 'display', 'handwriting', 'monospace', 'slab-serif', 'script', 'gothic', 'ornamental']
selected_fonts = []
min_fonts_per_category = 100 # Intentar obtener al menos 100 fuentes por categoría para alcanzar un total de 900

# Itero por categorías para asegurar diversidad
for category in font_categories:
    category_fonts = [f for f in all_fonts if f.get('category') == category]
    # Tomo las primeras `min_fonts_per_category` de cada categoría, o todas si hay menos
    selected_fonts.extend(category_fonts[:min_fonts_per_category])
    print(f"Añadidas {len(category_fonts[:min_fonts_per_category])} fuentes de la categoría '{category}'")

# Si no alcanzo las 900, añadir más de las restantes hasta llegar a la meta (priorizando las más populares)
if len(selected_fonts) < 900:
    remaining_fonts = [f for f in all_fonts if f not in selected_fonts]
    # Añado el resto de las fuentes hasta alcanzar al menos 900
    selected_fonts.extend(remaining_fonts[:(900 - len(selected_fonts))])
    print(f"Añadidas {len(selected_fonts)} fuentes en total (buscando al menos 900).")

if not selected_fonts:
    print("No se encontraron fuentes válidas para descargar. Revisa la configuración de la API o los filtros.")

# Descargo Fuentes y Recopilo Metadatos ---

print("\nIniciando descarga de fuentes y recolección de metadatos...")
font_dataset_records = []

# Uso la libreria tqdm para mostrar una barra de progreso mientras se procesan las fuentes
for font_info in tqdm(selected_fonts, desc="Procesando fuentes"):
    font_name = font_info['family']
    category = font_info.get('category', 'unknown') # Obtengo la categoría

    # Creo una subcarpeta para cada familia de fuente
    font_family_dir = os.path.join(OUTPUT_DIR, font_name.replace(" ", "_")) # Reemplazo espacios por guiones bajos
    os.makedirs(font_family_dir, exist_ok=True)

    # La API proporciona un diccionario 'files' con URLs para cada estilo/peso
    styles_available = font_info.get('files', {})

    for style_weight, url in styles_available.items():
        if url: # Me aseguraro de que la URL exista
            # Construyo el nombre del archivo TTF
            file_name = f"{font_name.replace(' ', '_')}-{style_weight}.ttf"
            file_path = os.path.join(font_family_dir, file_name)

            try:
                response = requests.get(url, stream=True)
                response.raise_for_status() # Lanza una excepción si la respuesta no es 200 OK

                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)

                # Añado registro al dataset
                font_dataset_records.append({
                    'font_name': font_name,
                    'style_weight': style_weight,
                    'file_path': file_path,
                    'category': category
                })
            except requests.exceptions.RequestException as e:
                print(f"Error descargando {font_name} ({style_weight}) de {url}: {e}")
            except Exception as e:
                print(f"Error procesando {font_name} ({style_weight}): {e}")

print(f"\nDescarga de fuentes completada. Se descargaron {len(font_dataset_records)} variantes de fuentes.")

# Creo y guardo el Dataset CSV

if font_dataset_records:
    df_fonts = pd.DataFrame(font_dataset_records)
    dataset_csv_path = os.path.join(OUTPUT_DIR, 'fonts_dataset.csv')
    df_fonts.to_csv(dataset_csv_path, index=False)
    print(f"\nDataset CSV guardado en: {dataset_csv_path}")
    print("\nPrimeras 5 filas del dataset:")
    print(df_fonts.head())
else:
    print("\nNo se pudo generar el dataset CSV ya que no se descargó ninguna fuente.")


Las fuentes se guardarán en: /content/drive/MyDrive/ocr_fonts
Directorio '/content/drive/MyDrive/ocr_fonts' asegurado.

Obteniendo lista de fuentes de Google Fonts (esto puede tardar unos segundos)...
Total de fuentes disponibles en la API: 1844
Añadidas 100 fuentes de la categoría 'sans-serif'
Añadidas 100 fuentes de la categoría 'serif'
Añadidas 100 fuentes de la categoría 'display'
Añadidas 100 fuentes de la categoría 'handwriting'
Añadidas 52 fuentes de la categoría 'monospace'
Añadidas 0 fuentes de la categoría 'slab-serif'
Añadidas 0 fuentes de la categoría 'script'
Añadidas 0 fuentes de la categoría 'gothic'
Añadidas 0 fuentes de la categoría 'ornamental'
Añadidas 900 fuentes en total (buscando al menos 900).

Iniciando descarga de fuentes y recolección de metadatos...


Procesando fuentes:   0%|          | 0/900 [00:00<?, ?it/s]


Descarga de fuentes completada. Se descargaron 3562 variantes de fuentes.

Dataset CSV guardado en: /content/drive/MyDrive/ocr_fonts/fonts_dataset.csv

Primeras 5 filas del dataset:
    font_name style_weight                                          file_path  \
0  42dot Sans          300  /content/drive/MyDrive/ocr_fonts/42dot_Sans/42...   
1  42dot Sans      regular  /content/drive/MyDrive/ocr_fonts/42dot_Sans/42...   
2  42dot Sans          500  /content/drive/MyDrive/ocr_fonts/42dot_Sans/42...   
3  42dot Sans          600  /content/drive/MyDrive/ocr_fonts/42dot_Sans/42...   
4  42dot Sans          700  /content/drive/MyDrive/ocr_fonts/42dot_Sans/42...   

     category  
0  sans-serif  
1  sans-serif  
2  sans-serif  
3  sans-serif  
4  sans-serif  
